# 커피지수 part_2

In [2]:
import pandas as pd
import numpy as np
from glob import glob
from tqdm.notebook import tqdm
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import folium
import json
from matplotlib import font_manager, rc
mpl.rcParams['axes.unicode_minus'] = False
plt.rc('font', family = 'Malgun Gothic')

In [3]:
starbucks = pd.read_csv('./data1/스타벅스.csv')
coffeebean = pd.read_csv('./data1/커피빈.csv')
ediya = pd.read_csv('./data1/이디야.csv')
paik = pd.read_csv('./data1/빽다방.csv')

In [4]:
# 고성군 - 고성(강원), 고성(경남)
# 광역시 - 서울 용산, 서울 서대문, 대전 서구, 대전 유성, 세종
# 행정구 - 수원장안, 용인 수지, 고양 일산동, 창원 합포, 창원 회원
tmp_gu_dict = {
    '수원' : ['장안구', '권선구', '팔달구', '영통구'],
    '성남' : ['수정구', '중원구', '분당구'],
    '안양' : ['만안구', '동안구'],
    '안산' : ['상록구', '단원구'],
    '고양' : ['덕양구', '일산동구', '일산서구'],
    '용인' : ['처인구', '기흥구', '수지구'],
    '청주' : ['상당구', '서원구', '흥덕구', '청원구'],
    '천안' : ['동남구', '서북구'],
    '전주' : ['완산구', '덕진구'],
    '포항' : ['남구', '북구'],
    '창원' : ['의창구', '성산구', '진해구', '마산합포구', '마산회원구']
    }

In [5]:
metro_list = ['서울특별시', '부산광역시', '대구광역시', '인천광역시', '대전광역시', '광주광역시', '울산광역시']

In [6]:
def get_ID(df):
    
    # 시네임 체크
    si_name = [None] * len(df)
    for i in df.index:
        if df.시도명[i] in metro_list:
            if len(df.시군구명[i]) == 2: #서울 중구, 대전 서구
                si_name[i] = df.시도명[i][:2] + ' ' + df.시군구명[i]
            else: # 서울 용산, 서울 서대문
                si_name[i] = df.시도명[i][:2] + ' ' + df.시군구명[i][:-1]
        else:
            si_len = len(df.시군구명[i].split())
            if si_len == 1:
                if df.시군구명[i][:-1] == '고성':
                    if df.시도명[i] == '강원도' :
                        si_name[i] = '고성(강원)'
                    else:
                        si_name[i] = '고성(경남)'
                elif df.시군구명[i].find('세종') == 0:
                    si_name[i] = '세종'
                else:
                    si_name[i] = df.시군구명[i][:-1] # 세종, 광명, 김포, 강릉
            else:
                admin_gu = df.시군구명[i].split()[-1]
                for key, value in tmp_gu_dict.items():
                    if admin_gu in value:
                        if len(admin_gu) == 2: # 포항 북구, 포항 남구
                            si_name[i] = key + ' ' + admin_gu
                        elif len(admin_gu) == 5: # 창원 합포, 창원회원
                            si_name[i] = key + ' ' + admin_gu[2:-1]
                        else: # 수원 팔달, 용인 지
                            si_name[i] = key + ' ' + admin_gu[:-1]
    return si_name

In [7]:
starbucks['ID'] = get_ID(starbucks)
coffeebean['ID'] = get_ID(coffeebean)
ediya['ID'] = get_ID(ediya)
paik['ID'] = get_ID(paik)
len(starbucks.ID.unique()), len(coffeebean.ID.unique()), len(ediya.ID.unique()), len(paik.ID.unique())

(191, 92, 238, 174)

In [8]:
# 모든 단위 도시를 합하면 합집합
city_set = set(starbucks.ID.unique()) | set(coffeebean.ID.unique()) | set(ediya.ID.unique()) | set(paik.ID.unique())
len(city_set)

244

In [9]:
# starbucks 만 있고, ediya / paik 가 없는곳
set(starbucks.ID.unique()) - set(ediya.ID.unique()) - set(paik.ID.unique())

{'계룡', '고성(강원)', '영양', '진도'}

In [10]:
# 브랜드별 매장수 계산
# 스벅
starbucks.groupby('ID')[['상호명']].count().head()

,상호명
ID,
가평,2
강릉,4
거제,6
경산,8
경주,7


In [11]:
pt_sb = starbucks.groupby('ID')[['상호명']].count()
pt_sb.reset_index(inplace=True)
pt_sb.rename(columns={'상호명':'스타벅스'}, inplace=True)
pt_sb.head()

,ID,스타벅스
0,가평,2
1,강릉,4
2,거제,6
3,경산,8
4,경주,7


In [12]:
pt_cb = coffeebean.groupby('ID')[['상호명']].count()
pt_cb.reset_index(inplace=True)
pt_cb.rename(columns={'상호명':'커피빈'}, inplace=True)
pt_cb.head()

,ID,커피빈
0,경주,2
1,고양 덕양,1
2,고양 일산동,1
3,고양 일산서,1
4,광명,1


In [13]:
pt_ed = ediya.groupby('ID')[['상호명']].count()
pt_ed.reset_index(inplace=True)
pt_ed.rename(columns={'상호명':'이디야'}, inplace=True)
pt_ed.head()

,ID,이디야
0,가평,9
1,강릉,17
2,강진,1
3,거제,9
4,거창,1


In [14]:
pt_bd = paik.groupby('ID')[['상호명']].count()
pt_bd.reset_index(inplace=True)
pt_bd.rename(columns={'상호명':'빽다방'}, inplace=True)
pt_bd.head()

,ID,빽다방
0,강릉,2
1,거창,1
2,경산,4
3,경주,1
4,고령,1


In [15]:
len(pt_sb), len(pt_cb), len(pt_ed), len(pt_bd)

(191, 92, 238, 174)

In [16]:
# merge로 커피 데이터 병합
cf = pd.merge(pt_ed, pt_sb, how='left')
cf.head()

,ID,이디야,스타벅스
0,가평,9,2.0
1,강릉,17,4.0
2,강진,1,NaN
3,거제,9,6.0
4,거창,1,NaN


In [17]:
cf = pd.merge(cf, pt_cb, how='left')
cf = pd.merge(cf, pt_bd, how='left')
cf.head()

,ID,이디야,스타벅스,커피빈,빽다방
0,가평,9,2.0,NaN,NaN
1,강릉,17,4.0,NaN,2.0
2,강진,1,NaN,NaN,NaN
3,거제,9,6.0,NaN,NaN
4,거창,1,NaN,NaN,1.0


In [18]:
cf.fillna(0, inplace=True)
cf.스타벅스 = cf.스타벅스.astype(int)
cf.이디야 = cf.이디야.astype(int)
cf.커피빈 = cf.커피빈.astype(int)
cf.빽다방 = cf.빽다방.astype(int)

In [19]:
cf.head()

,ID,이디야,스타벅스,커피빈,빽다방
0,가평,9,2,0,0
1,강릉,17,4,0,2
2,강진,1,0,0,0
3,거제,9,6,0,0
4,거창,1,0,0,1


In [20]:
cf['커피지수'] = (cf.스타벅스 + cf.커피빈) / (cf.이디야 + cf.빽다방)
cf.head()

,ID,이디야,스타벅스,커피빈,빽다방,커피지수
0,가평,9,2,0,0,0.222222
1,강릉,17,4,0,2,0.210526
2,강진,1,0,0,0,0.000000
3,거제,9,6,0,0,0.666667
4,거창,1,0,0,1,0.000000


In [21]:
cf.to_csv('./data1/커피지수.csv', index=False)